In [1]:
import netCDF4
import pandas
import datetime
import numpy

In [2]:
minCutoffFitLat = 45.
delTimeCutOffNrstPass = 45 # min
mlonDiffOtrEndCutoff = 50.
delLatCutoff = 2.
delCtimeCutoff = 60. #min

In [3]:
fileList = [ "../poes_n15_20130302_proc.nc",\
                "../poes_n18_20130302_proc.nc",\
                "../poes_m01_20130302_proc.nc",\
                "../poes_n19_20130302_proc.nc",\
                "../poes_m01_20130302_proc.nc",\
                "../poes_n16_20130302_proc.nc",\
                "../poes_n17_20130302_proc.nc" ]
poesAllEleDataDF = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_ele_flux", "sat"] )
poesAllProDataDF = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_pro_flux", "sat"] )
for f in fileList:
    print "reading file-->", f
    # read variable from the netCDF files
    poesRawData = netCDF4.Dataset(f)
    poesDF = pandas.DataFrame( poesRawData.variables['time'][:], columns=[ "timestamp" ] )
    poesDF['date'] = pandas.to_datetime(poesDF['timestamp'], unit='ms')
    poesDF["alt"] = poesRawData.variables['alt'][:]
    poesDF["aacgm_lat_foot"] = poesRawData.variables['aacgm_lat_foot'][:]

    poesDF["aacgm_lon_foot"] = poesRawData.variables['aacgm_lon_foot'][:]
    poesDF["MLT"] = poesRawData.variables['MLT'][:]
    # round of to 2 decimals
    poesDF['alt'] = [ round( x, 2 ) for x in poesDF['alt']]
    poesDF['aacgm_lat_foot'] = [ round( x, 2 ) for x in poesDF['aacgm_lat_foot']]
    poesDF['aacgm_lon_foot'] = [ round( x, 2 ) for x in poesDF['aacgm_lon_foot']]
    poesDF['MLT'] = [ round( x, 2 ) for x in poesDF['MLT']]
    # Add up the fluxes
    poesDF["ted_ele_total_flux"] = poesRawData.variables['ted_ele_tel0_flux_4'][:] +\
            poesRawData.variables['ted_ele_tel0_flux_8'][:] + \
            poesRawData.variables['ted_ele_tel0_flux_11'][:] + \
            poesRawData.variables['ted_ele_tel0_flux_14'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_4'][:] +\
            poesRawData.variables['ted_ele_tel30_flux_8'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_11'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_14'][:]
    poesDF["ted_pro_total_flux"] = poesRawData.variables['ted_pro_tel0_flux_4'][:] +\
            poesRawData.variables['ted_pro_tel0_flux_8'][:] + \
            poesRawData.variables['ted_pro_tel0_flux_11'][:] + \
            poesRawData.variables['ted_pro_tel0_flux_14'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_4'][:] +\
            poesRawData.variables['ted_pro_tel30_flux_8'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_11'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_14'][:]
    poesDF['log_ele_flux'] = [0. if x <= 0. else round( numpy.log10(x), 2 )\
                 for x in poesDF['ted_ele_total_flux']]
    poesDF['log_pro_flux'] = [0. if x <= 0. else round( numpy.log10(x), 2 )\
                 for x in poesDF['ted_pro_total_flux']]
    # the current satellite number
    poesDF["sat"] = f[-19:-17]
#     seperate out electron and proton flux and discard all zeros
    currPoesEleFluxDF = poesDF[poesDF["log_ele_flux"] > 0.][ ["timestamp",\
                     "date", "aacgm_lat_foot", "aacgm_lon_foot", "MLT",\
                     "log_ele_flux", "sat"] ].reset_index(drop=True)
    currPoesProFluxDF = poesDF[poesDF["log_pro_flux"] > 0.][ ["timestamp",\
                     "date", "aacgm_lat_foot", "aacgm_lon_foot", "MLT",\
                     "log_pro_flux", "sat"] ].reset_index(drop=True)
    
    
    poesAllEleDataDF = poesAllEleDataDF.append( currPoesEleFluxDF )
    poesAllProDataDF = poesAllProDataDF.append( currPoesProFluxDF )
    # now delete all the rows for prev DFs
    # we don't want to duplicate data
    poesDF = poesDF.drop( poesDF.index )
    currPoesEleFluxDF = currPoesEleFluxDF.drop( currPoesEleFluxDF.index )
    currPoesProFluxDF = currPoesProFluxDF.drop( currPoesProFluxDF.index )
# create a date and time columns
poesAllEleDataDF["dateStr"] = poesAllEleDataDF["date"].map(lambda x: x.strftime('%Y%m%d'))
poesAllEleDataDF["time"] = poesAllEleDataDF["date"].map(lambda x: x.strftime('%H%M'))
poesAllProDataDF["dateStr"] = poesAllProDataDF["date"].map(lambda x: x.strftime('%Y%m%d'))
poesAllProDataDF["time"] = poesAllProDataDF["date"].map(lambda x: x.strftime('%H%M'))

reading file--> ../poes_n15_20130302_proc.nc
reading file--> ../poes_n18_20130302_proc.nc
reading file--> ../poes_m01_20130302_proc.nc
reading file--> ../poes_n19_20130302_proc.nc
reading file--> ../poes_m01_20130302_proc.nc
reading file--> ../poes_n16_20130302_proc.nc
reading file--> ../poes_n17_20130302_proc.nc


In [4]:
timeRange = [ poesAllEleDataDF["date"].min(), poesAllEleDataDF["date"].max() ]
ctime = timeRange[0]
timeInterval=datetime.timedelta(minutes=30)
while ctime <= timeRange[1]:
    ctime += timeInterval
    if abs( ctime - datetime.datetime(2013,3,2,5) ) < datetime.timedelta(minutes=1):
        break
print ctime

2013-03-02 05:00:00.201000


In [5]:
# We only need those times when POES was above minCutoffFitLat(45) MLAT
poesAllEleDataDF = poesAllEleDataDF[ \
                ( abs( poesAllEleDataDF["aacgm_lat_foot"] ) >= minCutoffFitLat )\
                ].reset_index(drop=True)
# We only need a few columns, discard the rest
poesAllEleDataDF = poesAllEleDataDF[ [ 'sat', 'date',\
                        'aacgm_lat_foot', 'aacgm_lon_foot',\
                            'MLT', 'log_ele_flux' ] ]
poesAllEleDataDF["delCtime"] = abs(poesAllEleDataDF["date"] - ctime)
poesAllEleDataDF["delLatFit"] = abs( abs( poesAllEleDataDF["aacgm_lat_foot"] ) -\
                                    abs( minCutoffFitLat ) )
# We are sorting by sats, dates and lats to pick the nearest time
# when the satellite is between two 45 MLATs
poesAllEleDataDFNth = poesAllEleDataDF[ poesAllEleDataDF["aacgm_lat_foot"]\
                        >= 0. ].sort_values( ['sat', 'date', 'aacgm_lat_foot'],\
                                ascending=True ).reset_index(drop=True).drop_duplicates()
poesAllEleDataDFSth = poesAllEleDataDF[ poesAllEleDataDF["aacgm_lat_foot"]\
                        < 0. ].sort_values( ['sat', 'date', 'aacgm_lat_foot'],\
                                ascending=True ).reset_index(drop=True).drop_duplicates()

In [6]:
# Now we need to pick the satellite path
# which is closest to the selected time.!
# Northern Hemisphere
poesAllEleDataDFNthST = poesAllEleDataDFNth[ \
                            (poesAllEleDataDFNth["date"] \
                                >= ctime-datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass)) &\
                                (poesAllEleDataDFNth["date"] <=\
                                 ctime+datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass))].reset_index(drop=True)
poesAllEleDataDFNthST = poesAllEleDataDFNthST.sort_values(\
                            ["sat","date"], ascending=[True, True] ).reset_index(drop=True)
# We'll get the the satellite pass which is moving polewards
# Basically percent change in latitudes should be positive
# for a satellite moving polewards (percent change would help
# with the southern hemisphere lcoations.)
poesAllEleDataDFNthST["latRowDiffs"] = poesAllEleDataDFNthST.groupby("sat")[[\
                "aacgm_lat_foot" ] ].pct_change()
poesAllEleDataDFNthST = poesAllEleDataDFNthST[\
                        poesAllEleDataDFNthST["latRowDiffs"] > 0.\
                        ].reset_index(drop=True)
poesAllEleDataDFNthST = poesAllEleDataDFNthST.sort_values(\
                            ["sat", "aacgm_lat_foot","delCtime"]\
                            ).reset_index(drop=True)
# get the start time
selTimeRangeNthDF = poesAllEleDataDFNthST.groupby("sat").first().reset_index()
# Now if the time difference is too large, discard the satellite data
selTimeRangeNthDF = selTimeRangeNthDF[ selTimeRangeNthDF["delCtime"] <= \
                    datetime.timedelta(minutes=delTimeCutOffNrstPass)\
                    ].reset_index()

print selTimeRangeNthDF

   index sat                    date  aacgm_lat_foot  aacgm_lon_foot    MLT  \
0      0  01 2013-03-02 05:09:28.746           45.13          303.08  20.28   
1      1  16 2013-03-02 04:28:24.986           45.77          302.61  19.43   
2      2  17 2013-03-02 05:38:46.194           45.24          251.48  17.42   
3      3  18 2013-03-02 04:50:15.132           45.74          215.10  14.03   
4      4  19 2013-03-02 04:42:44.994           45.39          195.69  12.59   

   log_ele_flux        delCtime  delLatFit  latRowDiffs  
0          3.20 00:09:28.545000       0.13     0.002221  
1          3.35 00:31:35.215000       0.77     0.008594  
2          3.08 00:38:45.993000       0.24     0.004218  
3          2.62 00:09:45.069000       0.74     0.007045  
4          2.73 00:17:15.207000       0.39     0.007100  


In [7]:
print ctime-datetime.timedelta(minutes=delTimeCutOffNrstPass)

2013-03-02 04:15:00.201000
